# TITANIC

## Carregando Bibliotecas & Habilitando Scripts

In [1]:
import pandas as pd
import os
import numpy as np
import sys
# Adiciona o diretório 'Scripts' ao sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..','Scripts')))
from preprocessing import preprocess_data
from feature_engineering import engineer_features


## Carregando os dados

In [2]:
# Define o caminho relativo
file_path = os.path.join('..', 'Data', 'train.csv')  # '..' sobe um nível de diretório

# Carrega os dados
df = pd.read_csv(file_path)

## Verificação dos dados para o pré-processamento

In [3]:
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [6]:
df.duplicated().sum()

0


## Etapas a serem cumpridas no pré-processamento

* Preencher valores ausentes nas colunas `Age`, `Cabin` e `Embarked`.
* OHE na coluna `Sex` e `Embarked`
* Renomear as colunas para que a primeira letra seja minúscula e mantenha um padrão. 

#### Como Preencher: `Age`

In [7]:
dfna = df[df['Age'].isna()]
dfna

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
17,18,1,2,"Williams, Mr. Charles Eugene",male,NaN,0,0,244373,13.0000,NaN,S
19,20,1,3,"Masselmani, Mrs. Fatima",female,NaN,0,0,2649,7.2250,NaN,C
26,27,0,3,"Emir, Mr. Farred Chehab",male,NaN,0,0,2631,7.2250,NaN,C
28,29,1,3,"O'Dwyer, Miss. Ellen ""Nellie""",female,NaN,0,0,330959,7.8792,NaN,Q
...,...,...,...,...,...,...,...,...,...,...,...,...
859,860,0,3,"Razi, Mr. Raihed",male,NaN,0,0,2629,7.2292,NaN,C
863,864,0,3,"Sage, Miss. Dorothy Edith ""Dolly""",female,NaN,8,2,CA. 2343,69.5500,NaN,S
868,869,0,3,"van Melkebeke, Mr. Philemon",male,NaN,0,0,345777,9.5000,NaN,S
878,879,0,3,"Laleff, Mr. Kristo",male,NaN,0,0,349217,7.8958,NaN,S


* Irei utilizar groupby com caracteristicas para definir o mais presente nesses valores nan de `Age` e preencher a coluna.


In [8]:
x = df.groupby(['Pclass', 'Parch'])['Age'].mean().reset_index()
x['Age'].isna().sum()

0

* Ao agrupar `Pclass` e `Parch`, vemos que a media de idade irá ficar completa. Vamos fazer uma função lambda no script para completar os valores nan e ir simplificando a função até todos os valores serem completos.

### Como Preencher: `Cabin`

In [9]:
df['Cabin'].value_counts()

Cabin
B96 B98        4
G6             4
C23 C25 C27    4
C22 C26        3
F33            3
              ..
E34            1
C7             1
C54            1
E36            1
C148           1
Name: count, Length: 147, dtype: int64

In [10]:
dfc = df[~df['Cabin'].isna()]
dfc['Cabin'].unique()

array(['C85', 'C123', 'E46', 'G6', 'C103', 'D56', 'A6', 'C23 C25 C27',
       'B78', 'D33', 'B30', 'C52', 'B28', 'C83', 'F33', 'F G73', 'E31',
       'A5', 'D10 D12', 'D26', 'C110', 'B58 B60', 'E101', 'F E69', 'D47',
       'B86', 'F2', 'C2', 'E33', 'B19', 'A7', 'C49', 'F4', 'A32', 'B4',
       'B80', 'A31', 'D36', 'D15', 'C93', 'C78', 'D35', 'C87', 'B77',
       'E67', 'B94', 'C125', 'C99', 'C118', 'D7', 'A19', 'B49', 'D',
       'C22 C26', 'C106', 'C65', 'E36', 'C54', 'B57 B59 B63 B66', 'C7',
       'E34', 'C32', 'B18', 'C124', 'C91', 'E40', 'T', 'C128', 'D37',
       'B35', 'E50', 'C82', 'B96 B98', 'E10', 'E44', 'A34', 'C104',
       'C111', 'C92', 'E38', 'D21', 'E12', 'E63', 'A14', 'B37', 'C30',
       'D20', 'B79', 'E25', 'D46', 'B73', 'C95', 'B38', 'B39', 'B22',
       'C86', 'C70', 'A16', 'C101', 'C68', 'A10', 'E68', 'B41', 'A20',
       'D19', 'D50', 'D9', 'A23', 'B50', 'A26', 'D48', 'E58', 'C126',
       'B71', 'B51 B53 B55', 'D49', 'B5', 'B20', 'F G63', 'C62 C64',
       'E24

In [11]:
df['Cabin'].mode()

0        B96 B98
1    C23 C25 C27
2             G6
Name: Cabin, dtype: object

* A melhor forma de preencher `Cabin` sera com um dado novo 'Unk' (Unknown).

### Como Preencher: `Embarked`

In [12]:
df[df['Embarked'].isna()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
61,62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN
829,830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN


* A melhor forma de preencher `Emabrked` sera com um dado novo 'Unk' (Unknown).

## Pré-processamento

* Abaixo será carregado o script que contém todas as etapas do pré-processamento ditas anteriormentes 

In [13]:
# Usa a função
df = preprocess_data(df)

df

,passengerid,survived,pclass,name,age,sibsp,parch,ticket,fare,cabin,sex_male,embarked_c,embarked_q,embarked_s,embarked_unk
0,1,0,3,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,Unk,True,False,False,True,False
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,False,True,False,False,False
2,3,1,3,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,Unk,False,False,False,True,False
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,False,False,False,True,False
4,5,0,3,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,Unk,True,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",27.0,0,0,211536,13.0000,Unk,True,False,False,True,False
887,888,1,1,"Graham, Miss. Margaret Edith",19.0,0,0,112053,30.0000,B42,False,False,False,True,False
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",13.5,1,2,W./C. 6607,23.4500,Unk,False,False,False,True,False
889,890,1,1,"Behr, Mr. Karl Howell",26.0,0,0,111369,30.0000,C148,True,True,False,False,False


### Verificação se as etapas foram cumpridas

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   passengerid   891 non-null    int64  
 1   survived      891 non-null    int64  
 2   pclass        891 non-null    int64  
 3   name          891 non-null    object 
 4   age           891 non-null    float64
 5   sibsp         891 non-null    int64  
 6   parch         891 non-null    int64  
 7   ticket        891 non-null    object 
 8   fare          891 non-null    float64
 9   cabin         891 non-null    object 
 10  sex_male      891 non-null    bool   
 11  embarked_c    891 non-null    bool   
 12  embarked_q    891 non-null    bool   
 13  embarked_s    891 non-null    bool   
 14  embarked_unk  891 non-null    bool   
dtypes: bool(5), float64(2), int64(5), object(3)
memory usage: 74.1+ KB


In [15]:
df.isna().sum()

passengerid     0
survived        0
pclass          0
name            0
age             0
sibsp           0
parch           0
ticket          0
fare            0
cabin           0
sex_male        0
embarked_c      0
embarked_q      0
embarked_s      0
embarked_unk    0
dtype: int64

- [x] Preencher valores ausentes nas colunas `Age`, `Cabin` e `Embarked`.

- [x] OHE na coluna `Sex`

- [x] Renomear as colunas para que a primeira letra seja minúscula e mantenha um padrão. 

## Feature Engeneering

In [16]:
df

,passengerid,survived,pclass,name,age,sibsp,parch,ticket,fare,cabin,sex_male,embarked_c,embarked_q,embarked_s,embarked_unk
0,1,0,3,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,Unk,True,False,False,True,False
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,False,True,False,False,False
2,3,1,3,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,Unk,False,False,False,True,False
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,False,False,False,True,False
4,5,0,3,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,Unk,True,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",27.0,0,0,211536,13.0000,Unk,True,False,False,True,False
887,888,1,1,"Graham, Miss. Margaret Edith",19.0,0,0,112053,30.0000,B42,False,False,False,True,False
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",13.5,1,2,W./C. 6607,23.4500,Unk,False,False,False,True,False
889,890,1,1,"Behr, Mr. Karl Howell",26.0,0,0,111369,30.0000,C148,True,True,False,False,False


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   passengerid   891 non-null    int64  
 1   survived      891 non-null    int64  
 2   pclass        891 non-null    int64  
 3   name          891 non-null    object 
 4   age           891 non-null    float64
 5   sibsp         891 non-null    int64  
 6   parch         891 non-null    int64  
 7   ticket        891 non-null    object 
 8   fare          891 non-null    float64
 9   cabin         891 non-null    object 
 10  sex_male      891 non-null    bool   
 11  embarked_c    891 non-null    bool   
 12  embarked_q    891 non-null    bool   
 13  embarked_s    891 non-null    bool   
 14  embarked_unk  891 non-null    bool   
dtypes: bool(5), float64(2), int64(5), object(3)
memory usage: 74.1+ KB


### Etapas identificadas
* Mudar o nome de `sex_male` para `is_male` e com valores int ao invés de bool
* Criação de colunas de grupos como por exemplo `age_group` , `cabin_group` e `fare_group`
* Criação de colunas com base no vocativo/titulo
* Criação de colunas `is_alone`, `family_size`
* Interação entre Sexo e Classe
* Criação de colunas com base no tamanho e quantidade que determinado `ticket` aparece

In [18]:
df['fare'].describe()

count    891.000000
mean      32.204208
std       49.693429
min        0.000000
25%        7.910400
50%       14.454200
75%       31.000000
max      512.329200
Name: fare, dtype: float64

In [19]:
# Usa a função para aplicar o feature engineering ao DataFrame 
df = engineer_features(df)

# Mostra o DataFrame resultante
df


,passengerid,survived,pclass,name,age,sibsp,parch,ticket,fare,cabin,...,is_male,cabin_group,title,last_name,family_size,fare_group,is_alone,ticket_length,ticket_qtd,sex_pclass_interaction
0,1,0,3,Owen Harris,22.0,1,0,A/5 21171,7.2500,Unk,...,1,Unk,Mr,Braund,2,Low Fare,0,9,1,male_3
1,2,1,1,John Bradley (Florence Briggs Thayer),38.0,1,0,PC 17599,71.2833,C85,...,0,C,Mrs,Cumings,2,High Fare,0,8,1,female_1
2,3,1,3,Laina,26.0,0,0,STON/O2. 3101282,7.9250,Unk,...,0,Unk,Miss,Heikkinen,1,Medium-Low Fare,1,16,1,female_3
3,4,1,1,Jacques Heath (Lily May Peel),35.0,1,0,113803,53.1000,C123,...,0,C,Mrs,Futrelle,2,High Fare,0,6,2,female_1
4,5,0,3,William Henry,35.0,0,0,373450,8.0500,Unk,...,1,Unk,Mr,Allen,1,Medium-Low Fare,1,6,1,male_3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,Juozas,27.0,0,0,211536,13.0000,Unk,...,1,Unk,Rev,Montvila,1,Medium-Low Fare,1,6,1,male_2
887,888,1,1,Margaret Edith,19.0,0,0,112053,30.0000,B42,...,0,B,Miss,Graham,1,Medium-High Fare,1,6,1,female_1
888,889,0,3,"Catherine Helen ""Carrie""",13.5,1,2,W./C. 6607,23.4500,Unk,...,0,Unk,Miss,Johnston,4,Medium-High Fare,0,10,2,female_3
889,890,1,1,Karl Howell,26.0,0,0,111369,30.0000,C148,...,1,C,Mr,Behr,1,Medium-High Fare,1,6,1,male_1


In [20]:
df.columns

Index(['passengerid', 'survived', 'pclass', 'name', 'age', 'sibsp', 'parch',
       'ticket', 'fare', 'cabin', 'embarked_c', 'embarked_q', 'embarked_s',
       'age_group', 'is_male', 'cabin_group', 'title', 'last_name',
       'family_size', 'fare_group', 'is_alone', 'ticket_length', 'ticket_qtd',
       'sex_pclass_interaction'],
      dtype='object')

In [21]:
filtered_df = df[(df['embarked_c'] == 0) & (df['embarked_q'] == 0)&(df['embarked_s'] == 0)]
filtered_df

,passengerid,survived,pclass,name,age,sibsp,parch,ticket,fare,cabin,...,is_male,cabin_group,title,last_name,family_size,fare_group,is_alone,ticket_length,ticket_qtd,sex_pclass_interaction
61,62,1,1,Amelie,40.37221,0,0,113572,80.0,B28,...,0,B,Miss,Icard,1,High Fare,1,6,2,female_1
829,830,1,1,George Nelson (Martha Evelyn),40.37221,0,0,113572,80.0,B28,...,0,B,Mrs,Stone,1,High Fare,1,6,2,female_1


* Continuamos tendo embarked unk, quando os identificadores dos outros lugares são == 0

In [22]:
df['title'].unique()

array(['Mr', 'Mrs', 'Miss', 'Master', 'Don', 'Rev', 'Dr', 'Mme', 'Ms',
       'Major', 'Lady', 'Sir', 'Mlle', 'Col', 'Capt', 'the Countess',
       'Jonkheer'], dtype=object)

In [23]:
# Verificar a idade média e o tamanho da família para cada título
print(df.groupby('title')['family_size'].mean())


title
Capt            3.000000
Col             1.000000
Don             1.000000
Dr              1.571429
Jonkheer        1.000000
Lady            2.000000
Major           1.000000
Master          4.675000
Miss            2.263736
Mlle            1.000000
Mme             1.000000
Mr              1.441006
Mrs             2.528000
Ms              1.000000
Rev             1.333333
Sir             2.000000
the Countess    1.000000
Name: family_size, dtype: float64


*Vou fazer o mapeamento na parte de ML*

### Todas as etapas concluídas com sucesso!

* Baixando o df pré-processado com feature engineering para treinar os modelos de Machine Learning

In [24]:
file_path = os.path.join('..', 'Data', 'processed_data.csv')
df.to_csv(file_path, index=False)